# AgCl as first test case

First test runs comparing tableau based solver (numerical or analytical gradients and logX versus X solver) and PHREEQC.

In [9]:
% setup environment
% set path to equilibrium solver
txt=pwd;
n=length(txt);
rootpath=txt; rootpath=txt(1:n-19);
PHREEQCpath=[rootpath,'/runPHREEQC'];
addpath(PHREEQCpath);
JEQUILIBpath=[rootpath,'/JEQUILIB'];
addpath(JEQUILIBpath);
% clear data and number format
clear; format short e
% turn off warnings
warning off
%graphics_toolkit ("notebook"); 
%graphics_toolkit ("plotly"); 
%graphics_toolkit("gnuplot")

In [10]:
% MODEL USING TABLEAU (different gradients or log versus linear variables)
%graphics_toolkit("notebook")
%model with just inorganic speciation
%geochemical model
% input the constants and solution chemistry

flag1=2; %flag1 for concentration based search vector (1) or for log based search vector (2).
flag2=1; %flag2 for numerical derivatives (2) or analtyical derivatives (1)
%database=[]; 
flag3=0; % flag3 1 for show warnings. 0 no warnings
flag4=0; %if=1 solve tableau one line at a time to get a good initial guess (no solids). really challenging probs only
flag5=0; %if=1 use stored initial guess for including solids

AgT=1.003e-4; ClT=1.003e-4; pH=7; %for phreeqc would be 1.003e-4 because inputs should be as molality concentrations, using 0.997 kg/L as density at 25 °C

TOTALS=[AgT ClT];
TOTALS(TOTALS==0)=1e-16; % get rid of zero values, b/c div by zero error
%TOTALS=[inorganicTOTALS]; %TOTALS=inorganicTOTALS;
pH=pH; pe=20.75-pH; 

tic;
[Ag,Cl,AgCls,MASSERR]=AgCltableau(pH,pe,TOTALS',flag1,flag2,flag3,flag4,flag5);
tableautime=toc;

Agmasserror=MASSERR(1); Clmasserror=MASSERR(2);

In [11]:
% MODEL USING PHREEQC

minerals=[{'AgCls'}]; totalvector=[AgT; ClT; ClT]; totalnames=[{'Ag'}; {'Cl'}; {'Na'}]; speciesexport=[{'Ag+'}; {'Cl-'}];
%database=['llnl_nosolubleAgCl.dat']; 
acid=['NaOH']; show=0; T=25;
%database=['AgCl.dat']; 
%database=['simplestAgCl.dat']; 
database=['simplestAgCl_ideal.dat']; 
%database=['AgClwithcomplexes.dat']; 

tic
[solutionspeciesconcs, speciesnames, solidconcs, solidnames]=runPHREEQCv2noHA(T,pH,pe,totalnames,totalvector,minerals,speciesexport,database,show,acid);
PHREEQCtime=toc;

AgPHREEQC=solutionspeciesconcs(1); ClPHREEQC=solutionspeciesconcs(2); AgClsPHREEQC=solidconcs(1);
AgmasserrorPHREEQC=AgT-AgPHREEQC-AgClsPHREEQC; ClmasserrorPHREEQC=ClT-ClPHREEQC-AgClsPHREEQC;

In [12]:
% exact model

Ksp=10^-9.7453;

% check SI

IAP=AgT*ClT;

Agexact=AgT; Clexact=ClT; AgClsexact=0;

if IAP>=Ksp
a=1; b=ClT-AgT; c=-Ksp;
t=roots([a b c]); t=t(t>0); t=real(t(imag(t)==0));
Agexact=t; AgClsexact=AgT-Ag; Clexact=ClT-AgClsexact;
end


In [13]:
% compare models

Agcompare=[Ag AgPHREEQC*0.997 Agexact]
Clcompare=[Cl ClPHREEQC*0.997 Clexact]
AgClcompare=[AgCls AgClsPHREEQC*0.997 AgClsexact]
Agmasserrorcompare=[Agmasserror AgmasserrorPHREEQC*0.997]
Clmasserrorcompare=[Clmasserror ClmasserrorPHREEQC*0.997]
timecompare=[tableautime PHREEQCtime]

Agcompare,1,2,3
1,1.34076e-05,1.33673e-05,1.34076e-05


Clcompare,1,2,3
1,1.34076e-05,1.33673e-05,1.34076e-05


AgClcompare,1,2,3
1,8.68924e-05,8.66318e-05,8.68924e-05


Agmasserrorcompare,1,2
1,4.06576e-20,-1.35119e-20


Clmasserrorcompare,1,2
1,1.35525e-20,-1.35119e-20


timecompare,1,2
1,0.090523,0.0297351


In [14]:
%% Repeat PHREEQC 50 times and report times

% Number of runs
nRuns = 50;
% Vector to store the time of each run
timesPHREEQC = zeros(nRuns, 1);

% Declare variables where will save the result of the last run
AgPHREEQC_last   = 0;
ClPHREEQC_last   = 0;
AgClsPHREEQC_last= 0;
AgMassErr_last   = 0;
ClMassErr_last   = 0;

for i = 1:nRuns
    tic;
    [solutionspeciesconcs, speciesnames, solidconcs, solidnames] = ...
        runPHREEQCv2noHA(T, pH, pe, totalnames, totalvector, ...
                         minerals, speciesexport, database, show, acid);
    timesPHREEQC(i) = toc;
    
    % Each iteration results
    AgPHREEQC = solutionspeciesconcs(1);
    ClPHREEQC = solutionspeciesconcs(2);
    AgClsPHREEQC = solidconcs(1);

    % Calculation of mass errors for this run
    AgmasserrorPHREEQC = AgT - AgPHREEQC - AgClsPHREEQC;
    ClmasserrorPHREEQC = ClT - ClPHREEQC - AgClsPHREEQC;

    % Save as "last" run (overwrite it in each loop)
    AgPHREEQC_last    = AgPHREEQC;
    ClPHREEQC_last    = ClPHREEQC;
    AgClsPHREEQC_last = AgClsPHREEQC;
    AgMassErr_last    = AgmasserrorPHREEQC;
    ClMassErr_last    = ClmasserrorPHREEQC;
end

% Mean and standard deviation of the 50 times
meanTimePHREEQC = mean(timesPHREEQC);
stdTimePHREEQC  = std(timesPHREEQC);

% Display results
fprintf('\n========== PHREEQC executed %d veces ==========\n', nRuns);
fprintf('Last run:\n');
fprintf('  [Ag(aq)]    = %.6e M\n', AgPHREEQC_last*0.997);
fprintf('  [Cl(aq)]    = %.6e M\n', ClPHREEQC_last*0.997);
fprintf('  [AgCl(s)]   = %.6e M\n', AgClsPHREEQC_last*0.997);
fprintf('  Error mass Ag = %.4e\n', AgMassErr_last*0.997);
fprintf('  Error mass Cl = %.4e\n', ClMassErr_last*0.997);
fprintf('-----------------------------------------------\n');
fprintf('Mean time in 50 runs (ms) = %.6f\n', meanTimePHREEQC*1000);
fprintf('Standard Deviation (ms)            = %.6f\n', stdTimePHREEQC*1000);
fprintf('===============================================\n');


========== PHREEQC executed 50 veces ==========
Last run:
  [Ag(aq)]    = 1.336735e-05 M
  [Cl(aq)]    = 1.336735e-05 M
  [AgCl(s)]   = 8.663175e-05 M
  Error mass Ag = -1.3512e-20
  Error mass Cl = -1.3512e-20
-----------------------------------------------
Mean time in 50 runs (ms) = 19.634852
Standard Deviation (ms)            = 2.281608


In [15]:
% Tableau std and mean times

nRuns = 50;                % número de corridas
timesTableau = zeros(nRuns,1);

% Variables to save the solution of the last run
Ag_last   = 0;
Cl_last   = 0;
AgCls_last= 0;
massErr_last = [0 0];

for i = 1:nRuns
    tStart = tic;
    [Ag, Cl, AgCls, MASSERR] = AgCltableau(pH, pe, TOTALS', ...
                                          flag1, flag2, flag3, ...
                                          flag4, flag5);
    timesTableau(i) = toc(tStart);

    % saving the results of this run
    Ag_last    = Ag;
    Cl_last    = Cl;
    AgCls_last = AgCls;
    massErr_last = MASSERR;
end

% Calculation of mean and standard deviation of times
meanTimeTableau = mean(timesTableau);
stdTimeTableau  = std(timesTableau);

% Final information of the last run
Agmasserr_last = massErr_last(1);
Clmasserr_last = massErr_last(2);

% Display results
fprintf('\n========== Tableau Scott (AgCltableau) executed %d veces ==========\n', nRuns);
fprintf('Last run:\n');
fprintf('  [Ag(aq)]    = %.6e M\n', Ag_last*0.997);
fprintf('  [Cl(aq)]    = %.6e M\n', Cl_last*0.997);
fprintf('  [AgCl(s)]   = %.6e M\n', AgCls_last*0.997);
fprintf('  Error mass Ag = %.4e\n', Agmasserr_last*0.997);
fprintf('  Error masa Cl = %.4e\n', Clmasserr_last*0.997);
fprintf('--------------------------------------------------------------\n');
fprintf('Mean time in %d runs (ms) = %.6f\n', nRuns, meanTimeTableau*1000);
fprintf('Standard deviation (ms)            = %.6f\n', stdTimeTableau*1000);
fprintf('==============================================================\n');


========== Tableau Scott (AgCltableau) executed 50 veces ==========
Last run:
  [Ag(aq)]    = 1.336735e-05 M
  [Cl(aq)]    = 1.336735e-05 M
  [AgCl(s)]   = 8.663175e-05 M
  Error mass Ag = 4.0536e-20
  Error masa Cl = 1.3512e-20
--------------------------------------------------------------
Mean time in 50 runs (ms) = 84.062676
Standard deviation (ms)            = 2.073472


In [16]:
%% Snippet to parse the “End of Run after ... Seconds.” from out.txt and 
%% compute the mean and std of the internal times, skipping any NaNs.

nRuns = 50;
intTimes = zeros(nRuns, 1);

for i = 1:nRuns
    % (1) Run PHREEQC or your call that produces out.txt

    % (2) Parse the line "End of Run after X.XXXX Seconds." from out.txt
    [status, outStr] = system("grep 'End of Run after' out.txt");
    % outStr might look like "End of Run after X.XXXX Seconds."

    expr = 'after\s+([0-9]*\.[0-9]+|[0-9]+)\s+Seconds';
    tokens = regexp(outStr, expr, 'tokens', 'once');

    if ~isempty(tokens)
        intTimes(i) = str2double(tokens{1});
    else
        % If we don't find the line, store NaN
        intTimes(i) = NaN;
    end
end

% Filter out any NaN entries
validIdx = ~isnan(intTimes);
validTimes = intTimes(validIdx);

% Compute mean and std for the valid subset
meanIntTime = mean(validTimes);
stdIntTime  = std(validTimes);

fprintf('\nInternal PHREEQC time across %d runs:\n', nRuns);
fprintf('  Mean (ms) = %.6f\n', meanIntTime*1000);
fprintf('  Std  (ms) = %.6f\n\n', stdIntTime*1000);




Internal PHREEQC time across 50 runs:
  Mean (ms) = 3.093000
  Std  (ms) = 0.000000

